# Copy Classic Models DB from local MySQL Server to App SQLite DB


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import configparser
from urllib.parse import quote_plus   

In [3]:
# Load secrets from the configuration file
secrets_path = 'C:\\Users\\jbarcc\\Desktop\\Hobby\\CS - ML_Local\\webapp_sl_sales-dashboard\\secrets.ini'

config = configparser.ConfigParser()
config.read(secrets_path)

# --- MySQL Connection Details ---
db_config = config['mysql']
mysql_port = db_config.getint('port')
mysql_host = db_config['host']
mysql_database = db_config['database']
mysql_user = db_config['user']
mysql_password = db_config.get('password', raw = True)

# URL-Encode the password to account for special characters
# quote_plus() safely converts special characters like @, #, or & 
mysql_encoded_password = quote_plus(mysql_password)

In [4]:

# MySQL SQLAlchemy connection string
mysql_conn_str = f"mysql+mysqlconnector://{mysql_user}:{mysql_encoded_password}@{mysql_host}:{mysql_port}/{mysql_database}"
mysql_engine = create_engine(mysql_conn_str)

# --- SQLite Connection Details ---
# This will create the SQLite file in your current directory if it doesn't exist.
sqlite_file = config['sqlite']['file']
sqlite_engine = create_engine(f"sqlite:///../data/{sqlite_file}")

print(f"Connected to MySQL and SQLite file: {sqlite_file}")

Connected to MySQL and SQLite file: classicmodels.sqlite


In [5]:
# Function to get table names (may vary slightly based on MySQL version/configuration)
def get_mysql_tables(engine):
    query = "SHOW TABLES"
    # The output column name is usually the table name with 'Tables_in_{database}'
    return pd.read_sql(query, engine).iloc[:, 0].tolist()

table_names = get_mysql_tables(mysql_engine)
print(f"Tables found in MySQL: {table_names}")

Tables found in MySQL: ['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']


In [6]:
for table in table_names:
    print(f"Copying table: {table}...")
    try:
        # 1. Read the table data from MySQL into a pandas DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", mysql_engine)

        # 2. Write the DataFrame to the SQLite database
        # if_exists='replace' drops and recreates the table
        # index=False prevents writing the DataFrame index as a column
        df.to_sql(table, sqlite_engine, if_exists='replace', index=False)
        print(f"Successfully copied table: {table}")

    except Exception as e:
        print(f"Error copying table {table}: {e}")

print("\nDatabase copy process complete.")

Copying table: customers...
Successfully copied table: customers
Copying table: employees...
Successfully copied table: employees
Copying table: offices...
Successfully copied table: offices
Copying table: orderdetails...
Successfully copied table: orderdetails
Copying table: orders...
Successfully copied table: orders
Copying table: payments...
Successfully copied table: payments
Copying table: productlines...
Successfully copied table: productlines
Copying table: products...
Successfully copied table: products

Database copy process complete.
